<div style="text-align: right"> <h1>Carolyne Pelletier 101054962</h1></div>
<div style="text-align: right"> <h1>Akhil Dalal 100855466</h1></div>

# Question 1
## Feed Forward Neural Network

Develop a feed forward neural network in python that classifies the images found in the MNIST dataset.

## Architecture

The feed forward neural network consists of three layers. Each layer consists of a certain number of neurons. Note that this is a fully connected network.

### The Input Layer

The input layer takes in the greyscale values of each pixel in a 28x28 pixel image of a handwritten number. Each neuron takes one pixel as input, therefore, there are 784 neurons in the input layer.  
The pixel greyscale values range from 0 to 255. We normalize this to be between 0.0 and 1.0 by dividing each pixel's value by 255. This prevents any overflow when we use the sigmoid activation function.  
The input layer sends a weighted sum of its input to the next layer.

### The Hidden Layers

There can be multiple layers in the hidden layers. In our algorithm, we chose just a single hidden layer with 30 neurons.  
This layer gets the input from the input layer, and feeds it through the sigmoid activation function to get an output.  
The input for the next layer is the weighted sum of the output from the current layer with the weights to the next layer.

### The Output Layer

The output layer gets its inputs from the hidden layer. It puts the input through the sigmoid activation function to give the output for the network.  
The number of neurons in the output layer depends on the goal of the network. In our case, we need to classify 10 digits, so we are going to have 10 output neurons.  
The output is a 10x1 vector, where each element in the vector is the output of the respective neuron. The index holding the highest output value determines the prediction of the network.  
For example, if the highest value was in index 4, then the network classified our input as '4'.  
We update the weights and biases for the network based on this output.

## Algorithms

### Gradient Descent, Backpropagation and Weight Decay

Once we have our network set up, we must train the network to fine tune the weights. This, hopefully, results in a higher classification accuracy.  
**Note:** We measure accuracy as a percentage of correct classifications when the network is given some arbitrary amount of input (eg. 10000).  

We train the network using Stochastic Gradient Descent (SGD), and backpropagation.

```python
def stochastic_gradient_descent(self, training_set, learning_rate, epochs, lmbda):
        for i in range(epochs):
            shuffle(training_set)
            for tuple_ in training_set:
                self.update_weights(tuple_, len(training_set), learning_rate, lmbda)
```

Note that we shuffle the incoming training examples. This is done to prevent overfitting, since the network won't get the inputs in the same order in every epoch.  
The weights are updated after each training example. This is computationally intensive on large databases like the MNIST data.  
An improvement can be made by introducing mini-batches, where the network is trained on a small number of examples before updating the weights.

The SGD minimizes a cost function, in particular, the cross entropy cost function. A cost function measures how "far off" the network is with its prediction. Minimizing this gives us the best change in weights/biases needed to get a more accurate prediction.

The rule for updating the weights is:
$\large{w = w - \alpha \frac{\delta C}{\delta w}}$  

The rule for updating the biases is:
$\large{b = b - \alpha \frac{\delta C}{\delta b}}$  

where $C$ is the cost function, and $\alpha$ is the learning rate.

To find $\frac{\delta C}{\delta w}$ and $\frac{\delta C}{\delta b}$, we use backpropagation.

\newpage
```python
def backpropagation(self, image, label):

        bias_gradients = [np.zeros(b.shape) for b in self.biases]
        weight_gradients = [np.zeros(w.shape) for w in self.weights]

        # Feed Forward
        activations = []
        transfers = [image]

        for k in range(self.num_layers - 1):
            activations.append(np.dot(self.weights[k], transfers[k]) + self.biases[k])
            transfers.append(self.sigmoid(activations[k]))

        last_neuron_output = transfers[-1]

        # Back Propagation
        # Calculate the correction on last neuron
        last_neuron_error = self.cross_entropy_cost_prime(last_neuron_output, label)

        bias_gradients[-1] = last_neuron_error
        weight_gradients[-1] = np.dot(last_neuron_error, transfers[-2].transpose())

        # correct the other neurons in the hidden layers based on previous calculations
        hidden_neuron_error = last_neuron_error
        for k in range(2, self.num_layers):
            sp = self.sigmoid_prime(activations[-k])
            hidden_neuron_error = np.dot(self.weights[-k+1].transpose(), hidden_neuron_error) * sp
            bias_gradients[-k] = hidden_neuron_error
            weight_gradients[-k] = np.dot(hidden_neuron_error, transfers[-k-1].transpose())

        return (weight_gradients, bias_gradients)
```

Onces the gradients have been calculated, we update the weights and biases using the previously metioned rules.  
However, we can add a regularizer to the weights which helps in preventing over-fitting. This is known as the L2 regularizer, or weight decay. We scale the weights by some factor (a hyper-parameter) on every update.

When added to the cross entropy cost function, the rule for updating the weights is changed to 
$w = (1 - \frac{\alpha\lambda}{n})w - \alpha \frac{\delta C}{\delta w}$, where $\lambda$ is a hyper-parameter chosen by us. Usually, the higher the number of training examples, the higher the parameter.  
The bias update rule remains unchanged.

```python
 def update_weights(self, tuple_, training_set_size, learning_rate, lmbda):
        # call backpropagation on each tuple_
        image = tuple_[0]
        label = tuple_[1]
        weight_gradients, bias_gradients = self.backpropagation(image, label)

        # update weights and biases
        for k in range(len(self.weights)):
            self.weights[k] = (1 - ((learning_rate * lmbda) / training_set_size)) * self.weights[k] - (learning_rate * weight_gradients[k])

        for k in range(len(self.biases)):
            self.biases[k] -= learning_rate * bias_gradients[k]
```

### K-Fold Cross Validation

The MNIST database contains 70,000 images, out of which 60,000 are used for training, and 10,000 are used for testing. However, this is still insufficient data with which to train a network.  

One technique used it the $k$-fold cross validation. Here the data is split into $k$ mutually exclusive independent subsets, chosen at random.  
We iterate $k$ times, and on every iteration, we train on everything but the $k^{th}$ subset. That subset is used for testing. The average accuracy for the test is taken to be the final accuracy for the network.

```python
def k_fold_cross_validation(dataset, num_folds, layers, learning_rate, epochs, lmbda):
    #Split dataset into k mutually exclusive subsets (folds)
    folds = split_dataset(dataset.copy(), num_folds)
    fold_accuracy = []
    for i in range(num_folds):
        print("\nTraining with folds ", end="")
        training_folds = []

        # Combine all folds into a single training set
        for j in range(num_folds):
            if (i != j):
                print(j+1, " ", end="")
                training_folds.append(folds[j])
        print("")

        training_folds = sum(training_folds,[])
        testing_folds = folds[i]

        # initialize and train our network
        network = Network(layers)
        network.stochastic_gradient_descent(training_folds, learning_rate, epochs, lmbda)

        # test our network
        print("\nTesting with fold", i+1)
        fold_accuracy.append((network.network_accuracy(testing_folds)/len(testing_folds))*100)
        print("Classification Accuracy in Fold ", i+1, ": ", end="")
        print("%.3f%%" % fold_accuracy[i])
        print("________________________________________")

    return fold_accuracy

def split_dataset(dataset, num_folds):
    folds = []
    fold_size = int(len(dataset)/num_folds)
    for i in range(num_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset))
            fold.append(dataset.pop(index))
        folds.append(fold)

    return folds
```

## Network Accuracy

We use the following hyper-parameters to test varying hidden layer sizes:

```python
learning_rate = 0.1
epochs = 10
lmbda = 2.0
num_folds = 5
```

A few notes:

1. Too many hidden layers can cause the network to start learning slowly, and it might take longer to get to an optimal accuracy for the algorithm. We see this with 5 hidden layers, with 30 neurons each.

2. Too few hidden neurons, regardless of the number of hidden layers, may cause the network to not learn enough about the data. Therefore there will be poor prediction accuracy.

3. At a certain point, we do more computation (in turn take longer) for very little improvement in accuracy. We can see this when increase gradually from 15 to 75 neurons with a single hidden layer. The accuracy seems to plateau, however, the rate of improvement might increase.

4. Our network has a sweet spot: 1 hidden layer, 17-35 neurons. This way, we do not wait long to train, and we get consistent optimal result. (Empiracally, we've seen our network max out at 95-96% accuracy).

5. The hyper-parameters need to be tested for an even more optimal accuracy.

TO-DO: pretty sure we need to square # neurons in this table 

| Layers |   Neurons    | Network Accuracy | 
| ---------- | ------------- | ------------- |
| 1 | 2  |36.920%          
| 1 | 5  |77.590%            
| 1 | 15 | 92.780%           
| 1 | 30 | 94.720%           
| 1 | 45 | 95.930%           
| 1 | 75 | 96.710%
| 2 |  5 | 74.510%           
| 2 | 30 | 95.250%
| 5 | 30 | 94.750% 




\newpage
# Question 2

## RBF Network

Develop a feed forward RBF neural network in python that classifies the images found in the MNIST dataset.

## Architecture

A feed forward Radial Basis Function neural Network (RBFN) is a 3-layer neural network. It consists of a input layer, a single hidden layer, and an output layer.

Note that while this is a fully connected network, we do not use weights between the input and hidden layer.

### The Input Layer

As in the Feed Forward neural network, the input layer has 784 neurons, each representing the normalized greyscale values.

The input (a 784x1 vector) is given to each hidden layer neuron, without any weights, or weighted sums.

### The Hidden Layer

A neuron in this layer is also known as a RBF neuron. Each RBF neuron stores a prototype vector. An input is compared to this vector via the means of a gaussian function. Each neuron has a unique gaussian activation function, centered at the prototype vector.

The number of neurons in this layer can be determined by various methods. One such method is k-means clustering (discussed later). This method also gives a prototype vector.

### The Output Layer

The output layer outputs a "score" which is calculated by taking a weighted sum of the activation values from the hidden layer. The number of neurons in this layer is determined by the number of things we need to classify, in our case - 10 neurons.

We get a 10x1 vector as the output of the network, where the index with the highest value is our classification.

## Algorithms

### K-means Clustering

This algorithm is a way to determine the prototype vectors for the hidden RBF layer. By extension, it also determines the gaussian activation functions.

It works by randomly selecting $k$ points (centroids) from our data, clustering near-by points around the selection using the euclidean distance (clusters), and updating the centroids to the mean of their respective clusters.  
We keep doing this in a loop, using one of the following as our stopping condition:
1. No new point is assigned to any cluster (i.e. the centroids don't move)
2. For some maximum number of iterations
3. If the centroids don't update/move significantly determined by an arbitrary threshold.  

```python
# create clusters and update centroids till optimal centroids found
def k_means_clustering(k, data, iterations):

    # initialize centroids and starting clusters
    centroids = init_centroids(k, data)
    clusters = create_clusters(data, centroids)

    # loop update->create->update till no more change can be made to the centroids
    old_difference = 0
    i = 0
    while i < iterations:
        old_centroids = centroids
        centroids = update_centroids(clusters)
        clusters = create_clusters(data, centroids)

        # find out how much our centroids moved
        num_centroids = len(centroids)
        differences = []
        for i in range(num_centroids):
            differences.append(np.linalg.norm(old_centroids[i] - centroids[i]))

        max_diff = max(differences)

        difference_change = abs((max_diff-old_difference)/np.mean([old_difference,max_diff])) * 100

        if np.isnan(difference_change):
            break
        i+=1

    return (centroids,clusters)
```

The initial centroids are chosen randomly from our data. Note that we have a maximum number of iterations if the algorithm takes too long to converge. An empirical experience seems to suggest that with 70000 datapoints, it can take anywhere from 20-100 iterations to converge depending on the number of clusters/centroids.

### Gaussian Activation Function

```python
    def gaussian(self, x, mu, cluster):
        # calculate beta value
        sigma = 0.0
        for i in range(len(cluster)):
            sigma += np.linalg.norm(x - mu)

        sigma = sigma * (1.0/len(cluster))

        beta = 1.0 / (2*sigma**2)

        # calculate actual gaussian
        phi = np.exp(-beta*(np.linalg.norm(x - mu)**2))

        return phi
```

The gaussian activation function more commonly used is 
<center>$\large{\mathrm{e}^{-\beta(\|x - mu\|^{2})}}$  

To propagate the input forward, we calculate the gaussian activations for all RBF neurons, and then take a weighted sum.

## Network Accuracy

The accuracy results for our RBF network are too low to be meaningful (10-15%). A reason for this could be, the calculation for the $\beta$ values was taking too long so we could only use a 1000 datapoints for a pragmatic runtime.
This probably led to underfitting because of the very small amount of training datapoints.

\newpage
# Citations

1. A List of Cost Functions Used in Neural Networks, alongside Applications, stats.stackexchange.com/questions/154879/a-list-of-cost-functions-used-in-neural-networks-alongside-applications.  

2. Brownlee , Jason. machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/.  

3. Dobrzanski, Michael Daniel. Github, github.com/MichalDanielDobrzanski/DeepLearningPython35.  

4. Loeber, John. K-Means Clustering on Handwritten Digits, johnloeber.com/docs/kmeans.html.  

5. Mccormick, Chris. “Radial Basis Function Network (RBFN) Tutorial.” Radial Basis Function Network (RBFN) Tutorial · Chris McCormick, mccormickml.com/2013/08/15/radial-basis-function-network-rbfn-tutorial/.  

6. Nielsen , Michael. Neural Networks and Deep Learning, neuralnetworksanddeeplearning.com/.  

7. Roelants, Peter. “How to Implement a Neural Network Intermezzo 1.” Peter's Notes, peterroelants.github.io/posts/neural_network_implementation_intermezzo01/.  